## GPU General Information

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.client import device_lib

print(f"--- {tf.test.is_gpu_available()}\n")
print(f"--- {tf.test.gpu_device_name()}\n")
print(f"--- {tf.test.is_built_with_cuda()}\n")
print(f"---\n{device_lib.list_local_devices()}\n")

TensorFlow 1.x selected.
--- True

--- /device:GPU:0

--- True

---
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9553709449716076208
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 541715707723116681
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14126520263883718962
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5956749867016241656
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]



Previous execution some weeks ago showed that a **Tesla K80** was used instead.

```
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10550069380342432296
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16510589324407396482
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12770818163539961133
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276884378
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13215302358784350175
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
```

Followed by a **Tesla T4**.

 ```
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13999904789832453405
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2474607553434206280
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5105918168026004920
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800630580
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8208043236747855715
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
] 
```

## GPU Memory Availability Testing

Just to make sure that Google isn't purposely limiting access to the GPU memory.

In [2]:
# Source: https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# Only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printX():
    process = psutil.Process(os.getpid())
    print(f"Gen RAM Free: {humanize.naturalsize(psutil.virtual_memory().available)} | Proc size: {humanize.naturalsize(process.memory_info().rss)}")
    print(f"GPU RAM Free: {gpu.memoryFree:.0f} MB | Used: {gpu.memoryUsed:.0f} MB | Util {gpu.memoryUtil*100:.0f}% | Total {gpu.memoryTotal:.0f} MB")
printX()

Gen RAM Free: 12.6 GB | Proc size: 468.3 MB
GPU RAM Free: 14968 MB | Used: 111 MB | Util 1% | Total 15079 MB


## CPU General Information

The CPU appears to have two cores available.

In [3]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

## A basic Deep Learning Example #1

A text classification task on the IMDb Dataset.

In [4]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

word_index = imdb.get_word_index()
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

#print(train_data[0], type(train_data[0]))

# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
#model.add(keras.layers.Bidirectional(tf.keras.layers.LSTM(16)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=4,
                    batch_size=128,
                    validation_data=(x_val, y_val),
                    verbose=1)

results = model.evaluate(test_data, test_labels)

print(results)

Training entries: 25000, labels: 25000
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
Instructio

## A basic Deep Learning Example #2

A text classification task on the IMDb Dataset.

In [5]:
'''
#Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 128

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...


Using TensorFlow backend.


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 14s 576us/step - loss: 0.4803 - accuracy: 0.7710 - val_loss: 0.4145 - val_accuracy: 0.8135
Epoch 2/4
25000/25000 [==============================] - 14s 577us/step - loss: 0.3027 - accuracy: 0.8771 - val_loss: 0.3726 - val_accuracy: 0.8326
Epoch 3/4
25000/25000 [==============================] - 15s 590us/step - loss: 0.2329 - accuracy: 0.9116 - val_loss: 0.4107 - val_accuracy: 0.8274
Epoch 4/4
25000/25000 [==============================] - 3s 103us/step
Test score: 0.4442848878669739
Test accuracy: 0.8256400227546692


## A basic Deep Learning Example #3

An image classification task on the MNIST dataset.

In [6]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 4

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples

Train on 60000 samples, validate on 10000 samples




Epoch 1/4
60000/60000 [==============================] - 10s 169us/step - loss: 0.2677 - accuracy: 0.9170 - val_loss: 0.0574 - val_accuracy: 0.9819

Epoch 2/4
60000/60000 [==============================] - 5s 80us/step - loss: 0.0877 - accuracy: 0.9733 - val_loss: 0.0424 - val_accuracy: 0.9856
Epoch 3/4
60000/60000 [==============================] - 5s 80us/step - loss: 0.0657 - accuracy: 0.9804 - val_loss: 0.0313 - val_accuracy: 0.9892
Epoch 4/4
60000/60000 [==============================] - 5s 79us/step - loss: 0.0542 - accuracy: 0.9832 - val_loss: 0.0310 - val_accuracy: 0.9890
Test loss: 0.03100653666415019
Test accuracy: 0.9890000224113464


## A basic Deep Learning Example #4

A text classification task on the IMDb dataset, with an attached benchmark score for the K520 GPU for batch_size equal to 30.  
Refer to the 'GPU Specification List'.

In [7]:
'''
#Train a recurrent convolutional network on the IMDB sentiment classification task.
Gets to 0.8498 test accuracy after 2 epochs. 41 s/epoch on K520 GPU.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 4

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 31s 1ms/step - loss: 0.3876 - accuracy: 0.8157 - val_loss: 0.3208 - val_accuracy: 0.8631
Epoch 2/4
25000/25000 [==============================] - 30s 1ms/step - loss: 0.1942 - accuracy: 0.9250 - val_loss: 0.3729 - val_accuracy: 0.8533
Epoch 3/4
25000/25000 [==============================] - 29s 1ms/step - loss: 0.0944 - accuracy: 0.9684 - val_loss: 0.4419 - val_accuracy: 0.8468
Epoch 4/4
25000/25000 [==============================] - 9s 346us/step
Test score: 0.5678787619415671
Test accuracy: 0.8350399732589722


## Other Information - GPU Specifications List

| Model | Used in Tensorflow | Generation | Memory | Bus | Shading Units | GPU Clock | Memory Clock | Theoretical FP32 | Note |
| - | - | - | - | - | - | - | - | - | - |
| Tesla T4 | Yes | Tesla | 16 GB | 256bit | 2560 | 585MHz (1590 boosted) | 10000MHz effective | 8.141 TFLOPS | |
| Tesla  K80 | Yes | Tesla | 12 GB x2 | 384bit x2| 2496 x2 | 562MHz (824 boosted) | 5012MHz effective | 4.113 TFLOPS x2 | The 'x2' is irrelevant, only 1 is used |
| Grid K520 | No | Grid K5 | 4 GB x2 | 256bit x2 | 1536 x2 | 745MHz | 5000MHz effective | 2.289 TFLOPS x2 | The 'x2' is irrelevant, only 1 is used |